In [ ]:
import selenium

from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# from pywebcopy import save_webpage
import pyautogui
import random

import time
import matplotlib.pyplot as plt
import re
import cv2
import os

import numpy as np
import pandas as pd
import glob

In [1]:
from find_vehicle_image_urls import *

In [ ]:
url='https://www.autotrader.com/cars-for-sale/vehicle/736665996?city=Irvine&listingType=USED&makeCode=POR&modelCode=PORTAYCAN&referrer=%2Fporsche%2Ftaycan%3F&state=CA&zip=92604&clickType=listing'

driver = firefox_driver_init(headless=headless)

driver.get(url)

In [ ]:
from find_vehicle_image_urls import *

def chrome_driver_init(headless:bool=False, randomize:bool=True) -> webdriver.Firefox:
    options = webdriver.ChromeOptions()
    options.binary_location = chrome_path
    random_user_agent = create_random_user_agent()
    options.add_argument(f"user-agent={random_user_agent}")
    if headless:
        options.add_argument("--headless")

    # service = Service(chromedriver_path)
    # driver = webdriver.Chrome(service=service, options=options)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    return driver


driver = chrome_driver_init(headless=False)

In [ ]:
url='https://www.autotrader.com/cars-for-sale/ford/f150/san-diego-ca?firstRecord=50&searchRadius=0&sortBy=datelistedDESC&zip=92101'

driver = firefox_driver_init(headless=headless)

driver.get(url)
print(f"make/model search [{url}] webpage initiated\n")




In [ ]:
import webdriver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

chrome_path = "/Applications/chrome-mac-x64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"

options = webdriver.ChromeOptions()
options.binary_location = chrome_path

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.google.com")


In [ ]:
random_user_agent = create_random_user_agent()
options = webdriver.ChromeOptions()
options.binary_location = chrome_path
options.add_argument(f"user-agent={random_user_agent}")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.google.com")



In [2]:
from find_vehicle_image_urls import c2
url= 'https://www.autotrader.com/cars-for-sale/vehicle/736712659'
# driver = chrome_driver_init()
driver = c2()
driver.get(url)

In [ ]:
random_user_agent = create_random_user_agent()
options = webdriver.ChromeOptions()
options.binary_location = chrome_path
options.add_argument(f"user-agent={random_user_agent}")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.google.com")

In [ ]:
# time.sleep(wait_time) # be smarter and wait until things are actually loaded.
scrollbar_exists = wait_for_scrollbar(driver)
page_unavailable = check_for_site_unavailable(driver)
scrollbar_exists, page_unavailable

In [ ]:
## 'view all media' button to see the images
view_all_media_element = WebDriverWait(driver, wait_time).until(
    EC.visibility_of_element_located(
        (By.XPATH, "//p[normalize-space(text())='View All Media']")
    )
)
view_all_media_element.click()
print("view_all_media clicked successfully")


In [ ]:
scroll_panel = WebDriverWait(driver, wait_time).until(
    EC.visibility_of_element_located(
        (By.CSS_SELECTOR, "div[data-cmp='modalScrollPanel']")
    )
)
scroll_panel
image_urls = find_image_urls(driver)
last_length = len(image_urls)
i = 0
image_urls, last_length

In [ ]:
# import pandas as pd
#
# def find_image_urls_v2(driver) -> list:
#     # more flexible
#     image_url_prefix = 'https://images.autotrader.com/'
#     urls = list(filter(
#         lambda x: str(x).startswith(image_url_prefix),
#         str(driver.page_source).split()
#     ))
#     df_urls = pd.DataFrame({'url': urls})
#     df_urls['filename'] = df_urls['url'].apply(
#         lambda x: os.path.basename(x).split('.')[0]
#     )
#     # for each unique filename, keep the url w the largest image size e.g. 500
#     df_urls = df_urls.sort_values(by=['filename', 'url'], ascending=[1, 0])
#     df_urls.reset_index(drop=True, inplace=True)
#     df_urls2 = df_urls.groupby('filename').head(1)
#     # print(len(df_urls),len(df_urls2))
#     df_urls2
#     return list(df_urls2['url'])


In [ ]:
urls_so_far = []
urls_so_far += find_image_urls_v2(driver)
len(urls_so_far), urls_so_far[0]

In [ ]:
'''
<button data-cmp="closeButton" class="close" aria-label="Close" type="button"><span class="glyphicon glyphicon-remove"></span></button>
'''

In [ ]:
# yes
parent_element = driver.find_element(By.CSS_SELECTOR, 'div.modal')
close_button = parent_element.find_element(By.CSS_SELECTOR, '[data-cmp="closeButton"]')
close_button.click()


In [ ]:
# view more vehicles

'''
<a class="padding-left-0 text-nowrap margin-vertical-2 btn btn-link" href="/cars-for-sale/all-cars/toyota/tundra/san-diego-ca?zip=92101"><span aria-hidden="true" class="margin-right-1 glyphicon glyphicon-menu-left"></span>View more vehicles like this one</a>
'''

button = driver.find_element(By.LINK_TEXT, "View more vehicles like this one")
button.click()


In [ ]:

## scroll through all the photos


## try to scroll in a more naturalistic way    # scroll gradually until there all new image urls are captured
image_urls = find_image_urls(driver)
last_length = len(image_urls)
i = 0
while True:
    # Scroll down by a small amount
    driver.execute_script(f"arguments[0].scrollTop += {scroll_distance};", scroll_panel)
    # print(f'they see me scrollin, they hatin ({i})')
    i += 1
    time.sleep(scroll_pause_time)  # Wait for content to load
    image_urls = find_image_urls(driver)
    new_length = len(image_urls)
    if (new_length == last_length) or (i >= max_scrolls):  # If no more content is loading, break
        print(f'cant scroll anymore boss {i}')
        break
    last_length = new_length

return image_urls

In [ ]:

capture_listings_from_current_page


In [ ]:
vehicle_info= {'make': 'ford', 'model': 'taurus','zipcode':'92101','city_state_lower':'san-diego-ca'}
df,driver = find_listings_for_make_model(vehicle_info,quit=False)

In [ ]:

def capture_listings_from_current_page(driver) -> tuple:
    '''
    :param driver:
    :return: (pd.DataFrame, driver)
    '''

    # time.sleep(wait_time) # be smarter and wait until things are actually loaded.
    scrollbar_exists = wait_for_scrollbar(driver)
    page_unavailable = check_for_site_unavailable(driver)

    if (not scrollbar_exists) or page_unavailable:
        message = f'scrollbar failed to load {not scrollbar_exists} and/or page_unavailable {page_unavailable}; that is not good!'
        print(message)
        driver.quit()
        return pd.DataFrame(), None

    scroll_down_incrementally(driver)
    vehicle_listing_links = find_vehicle_listing_links(driver)

    ##  check if I got all the listings or not
    result_count_expected = get_search_result_count(driver)
    result_count_actual = len(vehicle_listing_links)
    _diff = result_count_expected - result_count_actual
    if _diff > 0:
        message = f'WARNING found {result_count_actual} / {result_count_expected} listings ; {_diff} are missing '
    else:
        # message = f'Found all {result_count_expected} listings'
        message = f'found {result_count_actual} / {result_count_expected} listings'
    print(message)

    current_url = driver.current_url

    df = pd.DataFrame(vehicle_listing_links, columns=['listing_header', 'url'])
    df = df.drop_duplicates()
    df['search_url'] = current_url
    search_timestamp = int(time.time())
    df['search_timestamp'] = search_timestamp
    # df['search_metadata'] = str(vehicle_info)
    print(df.head(1).T)
    return df, driver


In [ ]:
driver, capture_listings_from_current_page

In [ ]:
df_results_01, driver = capture_listings_from_current_page(driver)

In [ ]:
def _find_listings_for_make_model(vehicle_info:dict, driver=None, quit:bool=True) -> tuple:

    df_geog = load_geog_df()
    search_radius=0 # 0 corresponds to nationwide
    sort_by='distanceASC' # e.g. 'datelistedDESC','distanceASC'

    make, model = vehicle_info['make'], vehicle_info['model']
    first_record = int(vehicle_info.get('first_record', 0)) # 5000
    zipcode = vehicle_info.get('zipcode')
    city_state = vehicle_info.get('city_state_lower')
    if zipcode is None:
        print('choose random location within USA')
        location = list(df_geog.sample(n=1).to_dict('records'))[0]
        zipcode = location['zip']
        city_state = location['city_state_lower']


    ## you can do a lot by messing with the url, you know. really quite a bit. not everyone knows that.
    url_template = 'https://www.autotrader.com/cars-for-sale/' + \
        f'{make}/{model}/{city_state}?firstRecord={first_record}&searchRadius={search_radius}&sortBy={sort_by}&zip={zipcode}'
    # https://www.autotrader.com/cars-for-sale/bmw/3-series/houston-tx?firstRecord=5000&searchRadius=100&sortBy=distanceASC&zip=77038

    url = url_template.format(make=make, model=model,zipcode=zipcode)
    print(f"attempt to access webpage [{url}]")
    #

    ## initialize browser session - unless of course it is already initialized
    if driver is None:
        driver = firefox_driver_init(headless=headless)

    driver.get(url)
    print(f"make/model search [{url}] webpage initiated\n")

    # time.sleep(wait_time) # be smarter and wait until things are actually loaded.
    scrollbar_exists = wait_for_scrollbar(driver)

    page_unavailable = check_for_site_unavailable(driver)

    if (not scrollbar_exists) or page_unavailable:
        message=f'scrollbar failed to load {not scrollbar_exists} and/or page_unavailable {page_unavailable}; that is not good!'
        print(message)
        driver.quit()
        return pd.DataFrame(), None


    # print("make/model search - initiate scrolling")
    scroll_down_incrementally(driver)
    vehicle_listing_links = find_vehicle_listing_links(driver)

    ##  check if I got all the listings or not
    result_count_expected = get_search_result_count(driver)
    result_count_actual = len(vehicle_listing_links)
    _diff = result_count_expected - result_count_actual
    if _diff >0:
        message = f'WARNING found {result_count_actual} / {result_count_expected} listings ; {_diff} are missing '
    else:
        # message = f'Found all {result_count_expected} listings'
        message = f'found {result_count_actual} / {result_count_expected} listings'
    print(message)

    current_url = driver.current_url

    ## todo think about a way to navigate to page 2, page3 of search results
    # this can be done by clicking or it can be done by pre-populating the url in a certain way e.g. ?firstRecord=50
    # https://www.autotrader.com/cars-for-sale/ford/taurus/san-diego-ca?firstRecord=50&searchRadius=0&sortBy=distanceASC&zip=92101

    ## prepare result DF
    if quit:
        driver.quit()
        driver = None

    df = pd.DataFrame(vehicle_listing_links, columns=['listing_header', 'url'])
    df = df.drop_duplicates()
    df['search_url'] = current_url
    ts = int(time.time())
    df['make']= make
    df['model']= model
    df['search_timestamp'] =ts
    df['search_metadata'] = str(vehicle_info)
    print(df.head(1).T)
    filename =f'search_results_{make}_{model}_{ts}'
    filepath = f'{parent_directory_url_csvs}{filename}.csv'
    message=f'saving to {filepath}'
    print(message)
    df.to_csv(filepath, index=False, quoting=csv.QUOTE_ALL)

    return df, driver

In [ ]:
import datetime as dt
print(dt.datetime.now().replace(second=0, microsecond=0))

In [ ]:
vehicles = get_vehicle_make_model_list()
df = pd.DataFrame(vehicles)

print(df.sort_values(['body_style','make','model']) .set_index('body_style'))

In [ ]:
import os

directory = "/Users/levgolod/Projects/car_classifier/data/autotrader/vehicle_metadata/"

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)

        # Open file and count lines
        with open(file_path, "r") as f:
            lines = f.readlines()

        # If the file has only one line (header), delete it
        if len(lines) == 1:
            print(f"Deleting {filename} (Header-only CSV)")
            os.remove(file_path)


In [ ]:
vehicle_info= {
    'make': 'ford', 'model': 'taurus','zipcode':'92101','city_state_lower':'san-diego-ca',
    'first_record': 75
}
df2,driver = _find_listings_for_make_model(vehicle_info,driver=driver, quit=False)

In [ ]:
def load_geog_df() -> pd.DataFrame:
    '''
    Source https://simplemaps.com/data/us-zips
    :return:
    beverly-hills-ca
    '''
    # zips_filepath='~/Projects/car_classifier/data/simplemaps_uszips_basicv1.90/uszips.csv'
    zips_filepath='./data/simplemaps_uszips_basicv1.90/uszips.csv'
    zips_df = pd.read_csv(zips_filepath)
    zips_df['zip'] =zips_df['zip'].astype(str).str.zfill(5)
    zips_df['city_state_lower'] = zips_df['city'].str.lower().str.replace(' ','-') + '-' + \
                                  zips_df['state_id'].str.lower()
    return zips_df


df_geog = load_geog_df()
df_geog = df_geog.sample(frac=1)
print(df_geog[['zip','city_state_lower']].head(10))

In [ ]:
list(df_geog.sample(n=1).to_dict('records'))[0]

In [ ]:
url='https://www.autotrader.com/cars-for-sale/ford/f150/bloomington-id?firstRecord=5000&searchRadius=100&sortBy=distanceASC&zip=83223'
driver = firefox_driver_init(headless=headless)
driver.get(url)

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




wait_for_scrollbar(driver)

In [ ]:
def check_for_site_unavailable(driver):
    return 'site is currently unavailable' in str(driver.page_source)


check_for_site_unavailable(driver)

In [ ]:
## initialize browser session
url_cleaned='https://www.autotrader.com/cars-for-sale/vehicle/725617155'
driver = firefox_driver_init(headless=False)
driver.get(url_cleaned)
print(f"single vehicle listing [{url_cleaned}]- webpage initiated")



In [ ]:
'''
<div class="modal-header"><h4 id="modal-title" class="modal-title">Used 2022 Porsche Taycan<span class="text-normal"> - $60,944</span></h4><button data-cmp="closeButton" class="close" aria-label="Close" type="button"><span class="glyphicon glyphicon-remove"></span></button></div>
'''

In [ ]:
header_image= driver.find_element(By.CSS_SELECTOR, '[data-cmp="responsiveImage"]')
header_image
header_image.get_attribute('src')



In [ ]:
## create search results df

folder="/Users/levgolod/Projects/car_classifier/data/autotrader/vehicle_metadata/"
pattern=r'search_results_.*[0-9]*csv'
usecols=['listing_header',
 'url',
 'search_url',
 'make',
 'model',
 'search_timestamp',
 'search_metadata']
bigdf= pd.DataFrame(columns=usecols)


files = [x for x in os.listdir(folder) if bool(re.search(pattern, x))]

# file=files[0]
for file in files:
    print(file)
    try:
        df=pd.read_csv(folder+file, usecols=usecols)
    except ValueError:
        print(f'File {file} wrong columns, skipping')
        df=None

    if df is not None:
        bigdf = pd.concat([bigdf, df], ignore_index=True)
        bigdf=bigdf.drop_duplicates(subset=['url'])
        print(bigdf.shape)

# bigdf

# list(df)
# re.search('search_results_honda_civic_1739366121.csv', search_re)
# re.search(search_re, 'search_results_honda_civic_1739366121.csv')



In [ ]:
df.head()

In [ ]:
df, driver = find_listings_for_make_model(
    vehicle_info={'make':'ford','model':'taurus','zipcode':'90210'},
    quit=False
)

In [ ]:
## next add try/except
# the presence of this text bar "you have viewed X of Y results" tells you if you
# have the option to click to subsequent pages or not

driver
element = driver.find_element(By.CSS_SELECTOR, "div.results-text-container.text-size-300")
element, element.text

In [ ]:
# <div class="results-text-container text-size-300">You have viewed <span class="text-bold">25</span> of <span class="text-bold">195,802</span> Results</div>

# <button aria-label="Page 2" type="button" class="btn btn-default"><span class="sr-only"> Page</span>2</button>

button = driver.find_element(By.XPATH, "//button[@aria-label='Page 2']")
dir(button)
# button.click()
button.location_once_scrolled_into_view
button.click()

In [ ]:
driver.current_url

In [ ]:
from find_vehicle_image_urls import *

vehicles = [
        {'make':'bmw','model':'4-series'},
    {'make':'bmw','model':'3-series'},
    {'make':'bmw','model':'x5'},
    {'make':'bmw','model':'z4'},
    {'make':'ford','model':'taurus'},
    {'make':'ford','model':'f150'},
    {'make':'ford','model':'explorer'},
    {'make':'honda','model':'ridgeline'},
    {'make':'honda','model':'odyssey'},
    {'make':'honda','model':'accord'},
    {'make':'honda','model':'civic'},
    {'make':'honda','model':'passport'},
    {'make':'toyota','model':'camry'},
    {'make':'toyota','model':'crown'},
    {'make':'toyota','model':'tacoma'},
    {'make':'toyota','model':'tundra'},
    {'make':'toyota','model':'corolla'},
]

repeat=10
sleep=300
from timethis import timethis

for i in range(repeat):
    for vehicle in vehicles:
        message=f'iteration {i} vehicle {str(vehicle)}'
        print(message)
        try:
            with timethis(message):
                df, _ = find_listings_for_make_model(
                    vehicle_info=vehicle,
                    driver=None,
                    quit=True
                )
            print(message + ' success')
        except:
            print(message + ' failed')

        print(f'sleeping {sleep}')
        time.sleep(sleep)




In [ ]:
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

software_names = [SoftwareName.CHROME.value, SoftwareName.FIREFOX.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.MACOS.value]

ua = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)
random_user_agent = ua.get_random_user_agent()
print(random_user_agent)


In [ ]:
df, driver = find_listings_for_make_model(
    vehicle_info={'make':'ford','model':'taurus','zipcode':'90210'},
    driver=driver,
    quit=False
)

In [ ]:
!which geckodriver

In [ ]:
geckodriver_path = "/usr/local/bin/geckodriver"
parent_directory_url_csvs='~/Projects/car_classifier/data/autotrader/vehicle_metadata/'
wait_time=10

In [ ]:
make,model='porsche','taycan'
headless=False
url_template = 'https://www.autotrader.com/cars-for-sale/{}/{}'
url = url_template.format(make,model)


## initialize browser session
firefox_options = Options()
if headless:
    firefox_options.add_argument("--headless")

service = Service(geckodriver_path)
driver = webdriver.Firefox(service=service, options=firefox_options)
driver.get(url)
print("webpage initiated")

In [ ]:
def find_vehicle_listing_links(driver) -> list:
    '''
    return list of tuples
    [
         (
            'Certified 2021 Porsche Taycan 4 Cross Turismo',
            'https://www.autotrader.com/cars-for-sale/vehicle/737243275?listingType=USED&makeCode=POR&modelCode=PORTAYCAN&numRecords=25&referrer=%2Fcars-for-sale%2Fporsche%2Ftaycan&sortBy=relevance&zip=92101&clickType=spotlight'
        ),
    ]
    '''
    WebDriverWait(driver, wait_time).until(
        EC.visibility_of_element_located(
            (By.XPATH, '//a[starts-with(@href, "/cars-for-sale/vehicle/")]')
        )
    )
    elements = driver.find_elements(By.XPATH, '//a[starts-with(@href, "/cars-for-sale/vehicle/")]')
    return list(map(
        lambda element: (element.text, element.get_attribute("href")),
        elements
    ))

A = find_vehicle_listing_links(driver)
len(A),A[0]


In [ ]:
<select id="2281868035" class="form-control has-value" name="searchRadius"><option label="10 Miles" value="10">10 Miles</option><option label="25 Miles" value="25">25 Miles</option><option label="50 Miles" value="50">50 Miles</option><option label="75 Miles" value="75">75 Miles</option><option label="100 Miles" value="100">100 Miles</option><option label="200 Miles" value="200">200 Miles</option><option label="300 Miles" value="300">300 Miles</option><option label="400 Miles" value="400">400 Miles</option><option label="500 Miles" value="500">500 Miles</option><option label="Nationwide" value="0">Nationwide</option></select>

In [ ]:
df = pd.DataFrame(A, columns=['listing_header', 'url'])
df

In [ ]:
dropdown = Select(driver.find_element(By.NAME, "searchRadius"))
dropdown, dropdown.options
dropdown.select_by_value("100")
dropdown.select_by_value("0")

In [ ]:
dir(dropdown.options[-1])
dropdown.options[-1].text, dropdown.options[-1].tag_name

In [ ]:
selenium.common.exceptions.NoSuchElementException

In [ ]:
# scroll_panel = WebDriverWait(driver, wait_time).until(
#         EC.visibility_of_element_located(
#             (By.CSS_SELECTOR, "div[data-cmp='modalScrollPanel']")
#         )
#     )
# scroll_panel
driver.execute_script(f"window.scrollBy(0, {scroll_distance});")

image_urls = find_vehicle_listing_links(driver)
last_length =len(image_urls)
last_length

In [ ]:
vehicle_listing_links = find_vehicle_listing_links(driver)
last_length =len(image_urls)
i=0
while True:
    # Scroll down by a small amount
    driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
    print(f'they see me scrollin, they hatin ({i})')
    i+=1
    time.sleep(scroll_pause_time)  # Wait for content to load
    vehicle_listing_links = find_vehicle_listing_links(driver)
    new_length = len(vehicle_listing_links)
    if (new_length == last_length) or (i>=100) :  # If no more content is loading, break
        print('cant scroll anymore boss')
        break
    last_length = new_length

In [ ]:
# <h2 data-cmp="subheading" class="text-bold text-size-400 link-unstyled" role="heading" aria-level="3">Certified 2021 Porsche Taycan 4 Cross Turismo</h2>
# elements = driver.find_elements(By.CSS_SELECTOR, '[data-cmp="subheading"].text-bold.text-size-400.link-unstyled[role="heading"]')
# len(elements), elements[0],elements[0].text

elements = driver.find_elements(By.XPATH, '//a[starts-with(@href, "/cars-for-sale/vehicle/")]')
len(elements), elements[0], elements[0].text, elements[0].get_attribute("href")

# Extract and print the URLs
# urls = [element.get_attribute("href") for element in elements]
# print(urls)



In [ ]:
elements[0].click()

In [ ]:
<a data-cmp="link" class="link-undecorated" rel="sponsored,nofollow" data-next-link="true" href="/cars-for-sale/vehicle/736665996?city=Irvine&amp;listingType=USED&amp;makeCode=POR&amp;modelCode=PORTAYCAN&amp;referrer=%2Fporsche%2Ftaycan%3F&amp;state=CA&amp;zip=92604&amp;clickType=spotlight"><h2 data-cmp="subheading" class="text-bold text-size-400 link-unstyled" role="heading" aria-level="3">Used 2021 Porsche Taycan 4S</h2></a>

### Make sure I can launch firefox

In [ ]:

# Set up Firefox options (optional, e.g., for headless mode)
firefox_options = Options()
# Uncomment the next line if you want to run Firefox in headless mode (i.e., without a GUI)
# firefox_options.add_argument("--headless")

# If geckodriver is in your PATH, you can just initialize the driver.
# Otherwise, specify the path to the geckodriver executable:
geckodriver_path = "/usr/local/bin/geckodriver"  # Adjust if needed

service = Service(geckodriver_path)

# Initialize the Firefox driver
driver = webdriver.Firefox(service=service, options=firefox_options)

# Navigate to a website (replace with your target URL)
# driver.get("https://www.npr.com")
# url='https://www.autotrader.com/cars-for-sale/vehicle/739471281'
url='https://www.autotrader.com/cars-for-sale/vehicle/736665996'
driver.get(url)

# Wait a few seconds to let the page load
time.sleep(5)

# Example: Find all images on the page
# images = driver.find_elements("tag name", "img")
# print(f"Found {len(images)} images on the page.")

# Always quit the browser when done
# driver.quit()

### Try downloading the images for one car URL

In [ ]:
def clean_vehicle_url(url:str) -> str:
    pattern = r'^(https://.*?/vehicle/[a-zA-Z0-9]*)(\?.*)?$'
    re_result = re.match(pattern, str(url))
    if bool(re_result):
        return re_result.group(1)
    else:
        return None

In [ ]:
url='https://www.autotrader.com/cars-for-sale/vehicle/739471281?LNX=SPGOOGLEBRANDPLUSMAKE&city=San%20Diego&ds_rl=1289689&gad_source=1&gclid=CjwKCAiA5Ka9BhB5EiwA1ZVtvHjCtPelBybjmSVqEfXhXQ4FoLUB_-DHe9sxqf7bMrntLKD3J1AJKRoCz-8QAvD_BwE&gclsrc=aw.ds&listingType=USED&makeCode=TOYOTA&modelCode=CAMRY&referrer=%2Ftoyota%2Fcamry%3FLNX%3DSPGOOGLEBRANDPLUSMAKE%26ds_rl%3D1289689%26gad_source%3D1%26gclid%3DCjwKCAiA5Ka9BhB5EiwA1ZVtvHjCtPelBybjmSVqEfXhXQ4FoLUB_-DHe9sxqf7bMrntLKD3J1AJKRoCz-8QAvD_BwE%26gclsrc%3Daw.ds%26utm_campaign%3Dat_na_na_national_evergreen_roi_na_na%26utm_content%3Dkeyword_text_na_na_na_spgooglebrandplusmake_na%26utm_medium%3Dsem_brand-plus_perf%26utm_source%3DGOOGLE%26utm_term%3Dautotrader%2520toyota%2520camry&state=CA&utm_campaign=at_na_na_national_evergreen_roi_na_na&utm_content=keyword_text_na_na_na_spgooglebrandplusmake_na&utm_medium=sem_brand-plus_perf&utm_source=GOOGLE&utm_term=autotrader%20toyota%20camry&zip=92101&clickType=listing'
print(clean_vehicle_url(url))

In [ ]:
url_cleaned = clean_vehicle_url(url)
firefox_options = Options()
# Uncomment the next line if you want to run Firefox in headless mode (i.e., without a GUI)
# firefox_options.add_argument("--headless")


service = Service(geckodriver_path)
driver = webdriver.Firefox(service=service, options=firefox_options)
# Navigate to a website (replace with your target URL)
driver.get(url_cleaned)

view_all_media_element = WebDriverWait(driver, wait_time).until(
        EC.visibility_of_element_located(
            (By.XPATH, "//p[normalize-space(text())='View All Media']")
        )
    )

    # Click the element once it is clickable.
view_all_media_element.click()

print("Element clicked successfully!")

# <p class="margin-0">View All Media</p>

In [ ]:


page_source= driver.page_source
# print(str(page_source)[:1000])
'modalScrollPanel' in str(page_source)
#  <div data-cmp="modalScrollPanel" class="modal-scroll-panel" tabindex="0">
# driver.find_element("css selector", ".media-panel")
scroll_panel = driver.find_element("css selector", "div[data-cmp='modalScrollPanel']")
scroll_panel
driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scroll_panel)


In [ ]:
scroll_panel2 = WebDriverWait(driver, wait_time).until(
        EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "div[data-cmp='modalScrollPanel']")
        )
    )
scroll_panel2
# driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scroll_panel2)

driver.execute_script("arguments[0].scrollTop += 100;", scroll_panel2)


# can verify I'm at the bottom by looking for text "Estimated Finance Payment"

In [ ]:
page_source= driver.page_source
# print(str(page_source)[:1000])
'7dd47a88f70b4489910be87730faa5ba' in str(page_source)
urls = list(filter(
    lambda x: str(x).startswith('https://images.autotrader.com/scaler/500'),
    str(page_source).split()
))
print(len(urls))

In [ ]:
vehicle_id = url_cleaned.split('/')[-1]


download_folder = f"~/Downloads/{vehicle_id}/"  # Change this to your preferred save path
os.makedirs(download_folder, exist_ok=True)
# save_webpage(url_cleaned, download_folder) #lxml_html_clean


In [ ]:
EC.visibility_of_element_located(
            (By.XPATH, "//p[normalize-space(text())='View All Media']")
        )

elements = driver.find_elements(By.XPATH, "//p[normalize-space(text())='View All Media']")
len(elements) > 0

In [ ]:
elements[0].click()
# ElementClickInterceptedException: Message: Element <p class="margin-0"> is not clickable at point (912,330) because another element <div id="onetrust-button-group-parent" class="ot-sdk-three ot-sdk-columns has-reject-all-button"> obscures it
# Stacktrace:
# <a href="#" id="tabNavBar-tab-0" role="tab" aria-controls="tabNavBar-pane-0" aria-selected="true">Photos</a>
#

In [ ]:
'Gas Engine' in str(page_source) #todo extract this information as well

In [ ]:
scroll_pause_time = 0.5  # Adjust based on load time

# Get initial scroll height
last_height = driver.execute_script("return document.body.scrollHeight", scroll_panel2)
i=0
while True:
    # Scroll down by a small amount
    # driver.execute_script("window.scrollBy(0, 1000);", scroll_panel2)  # Scroll down in steps
    driver.execute_script("arguments[0].scrollTop += 500;", scroll_panel2)
    print(f'they see me scrollin, they hatin ({i})')
    i+=1
    time.sleep(scroll_pause_time)  # Wait for content to load

    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight", scroll_panel2)
    if new_height == last_height:  # If no more content is loading, break
        break
    last_height = new_height


In [ ]:
def find_image_urls(driver) -> list:
    # get the image urls. There are multiple sizes available but due to inflation I am gonna 'keep it 500'
    image_url_prefix = 'https://images.autotrader.com/scaler/500'
    return list(filter(
        lambda x: str(x).startswith(image_url_prefix),
        str(driver.page_source).split()
    ))


In [ ]:
# <h1 data-cmp="heading" id="vehicle-details-heading" class="text-ultra-bold text-size-400 text-size-sm-500 text-size-md-650">Used 2022 Toyota Camry SE</h1>

# Find the <h1> element
heading_element = driver.find_element("css selector", 'h1[data-cmp="heading"]#vehicle-details-heading')

# Extract the text
heading_text = heading_element.text
heading_text

In [ ]:
## capture year make model as text
year_make_model_element = WebDriverWait(driver, wait_time).until(
        EC.visibility_of_element_located(
            (By.CSS_SELECTOR, 'h1[data-cmp="heading"]#vehicle-details-heading')
        )
    )

year_make_model_element, year_make_model_element.text

In [ ]:
list(filter(
    lambda x: 'listColumns' in str(x),
    str(driver.page_source).split()
))

len(str(driver.page_source).split('''<ul data-cmp="listColumns" class="list columns columns-1'''))



In [ ]:
# list_elements = driver.find_elements(By.CSS_SELECTOR, 'ul[data-cmp="listColumns"]')
# len(list_elements)

list_element =listing_detail_element= driver.find_element(By.CSS_SELECTOR, 'ul[data-cmp="listColumns"].list.columns.columns-1')
# len(list_element)
list_element, list_element.text
listing_details = list_element.text.split('\n')
listing_details
listing_detail = listing_detail_element.text
listing_detail_clean = '|'.join(listing_detail.strip().replace('|',' ').split('\n'))
listing_detail_clean
# listing_details = str(list_element.text).split('\n')
# listing_details




In [ ]:
def clean_text_remove_newline(x:str) -> str:
    return '|'.join(str(x).strip().replace('|',' ').split('\n'))


In [ ]:
# <div class="text-size-400 display-flex justify-content-between align-items-center padding-vertical-3" data-cmp="listingPrice">Listing Price<span class="margin-left-auto">$25,488</span></div>

list_price_element = driver.find_element(By.CSS_SELECTOR, '[data-cmp="listingPrice"]')
list_price_element, list_price_element.text

list_price_clean = clean_text_remove_newline(list_price_element.text)
list_price_clean = list_price_clean.split('|')[-1].strip()
list_price_clean

# <span class="display-block display-sm-inline-block">VIN:&nbsp;4T1G11AK5NU020242</span>

In [ ]:
# elements = driver.find_elements(By.CSS_SELECTOR, '.display-block.display-sm-inline-block')
elements = driver.find_elements(By.CSS_SELECTOR, 'span.display-block.display-sm-inline-block')

elements, len(elements)
elements[0].text

def get_clean_vin(vin_text:str) -> str:
    '''
    'VIN: 4T1G11AK5NU020242'
    :param vin_text:
    :return:
    '''
    vin = str(vin_text).upper().split(':')[-1].strip()
    if len(vin) ==17:
        return vin
    else:
        return None

elements[0].text, get_clean_vin(elements[0].text)

In [ ]:
listing_narrative_element = driver.find_element(By.CSS_SELECTOR, '[data-cmp="seeMore"]')
listing_narrative_element, str(listing_narrative_element.text)[:1000]

In [ ]:
import csv
csv.QUOTE_ALL
df = pd.DataFrame({'vehicle_image_url': urls})
df['vehicle_id']= vehicle_id
filepath = f'{parent_directory_url_csvs}{vehicle_id}.csv'
message=f'saving to {filepath}'
print(message)
df.to_csv(filepath, index=False, quoting=csv.QUOTE_ALL)

In [ ]:
df2 = pd.read_csv(filepath)
df2 = pd.read_csv('~/Projects/car_classifier/data/autotrader/vehicle_metadata/736665996.csv')
df2

In [ ]:
special_chars = '|^'
internal_delim = '^'
chars_to_remove = sorted(set(special_chars + internal_delim))
chars_to_remove

In [ ]:
def clean_text_remove_newline(x:str) -> str:
    special_chars = '|^'
    internal_delim = '^'
    chars_to_remove = sorted(set(special_chars + internal_delim))
    x2 = str(x).strip()
    for char in chars_to_remove:
        x2 = x2.replace(char, ' ')

    lines= [y.strip() for y in x2.strip().split('\n')]
    return internal_delim.join(lines)

clean_text_remove_newline(df2['listing_details'][0])
clean_text_remove_newline(listing_detail)

In [ ]:
!wget -q -nc -P "/Users/levgolod/Projects/car_classifier/data/autotrader/vehicle_images/" "https://images.autotrader.com/scaler/500/375/hn/c/ff42e26fdabb4cf2bee049d3909ffee3.jpg"

In [ ]:
## I fw random proxies ; https://www.proxynova.com/proxy-server-list/
# 188.112.179.204	80	18 hours ago
# 396 ms	76% (129)	lv Latvia - Riga	Elite

from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType

# Set up proxy details
proxy = "98.176.105.96:3128"  # e.g., "123.456.789.101:8080"


# # Set up Firefox options to use the proxy
# firefox_profile = Options()
# firefox_profile.set_preference("network.proxy.type", 1)  # 1 for manual configuration
# firefox_profile.set_preference("network.proxy.http", proxy.split(":")[0])
# firefox_profile.set_preference("network.proxy.http_port", int(proxy.split(":")[1]))
# firefox_profile.set_preference("network.proxy.ssl", proxy.split(":")[0])
# firefox_profile.set_preference("network.proxy.ssl_port", int(proxy.split(":")[1]))
# service = Service(geckodriver_path)
# driver = webdriver.Firefox(service=service, options=firefox_profile)
# driver.get("https://www.whatismyip.com/")  # Check your IP address here

firefox_options = Options()
firefox_options.set_preference("network.proxy.type", 1)  # Enables manual proxy configuration
firefox_options.set_preference("network.proxy.http", proxy.split(":")[0])  # Proxy IP
firefox_options.set_preference("network.proxy.http_port", int(proxy.split(":")[1]))  # Proxy Port
firefox_options.set_preference("network.proxy.ssl", proxy.split(":")[0])  # HTTPS Proxy
firefox_options.set_preference("network.proxy.ssl_port", int(proxy.split(":")[1]))  # HTTPS Port
firefox_options.set_preference("network.proxy.socks_version", 5)  # Use SOCKS5 if needed

# Launch Firefox with the configured proxy
driver = webdriver.Firefox(options=firefox_options)

# Test if the proxy is working
driver.get("https://www.whatismyip.com/")


In [ ]:
text='Shop for a Car'
element = driver.find_element(By.XPATH, f"//*[contains(text(), '{text}')]")
element, element.is_displayed()

In [ ]:


# Example usage:
print(f"Scrolled: {get_scroll_percentage(driver)}%")


In [ ]:
max_scrolls=100
result_count_element = driver.find_element(By.CSS_SELECTOR, '[data-cmp="resultsCount"]')
result_count_element, result_count_element.text

def get_search_result_count(driver) -> int:
    result_count_element = driver.find_element(By.CSS_SELECTOR, '[data-cmp="resultsCount"]')
    result_count_text = result_count_element.text
    result_count = int(result_count_text.split(' ')[0])
    return result_count

get_search_result_count(driver)

In [ ]:
import time
ts = int(time.time())

import datetime as dt
ts, dt.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:


zips = load_zipcodes()
zips[:10], len(zips),len(set(zips)), random.choice(zips)

In [ ]:
scroll_down_incrementally(driver, scroll_distance, verbose=True)

In [ ]:
## get learned how to scroll more better



def scroll_until_text_visible(text, max_scrolls=20, scroll_increment=200):
    """Scrolls down bit-by-bit until the text is visible on screen."""
    scroll_attempts = 0
    while scroll_attempts < max_scrolls:
        try:
            # Locate the element (even if it's hidden)
            element = driver.find_element(By.XPATH, f"//*[contains(text(), '{text}')]")

            # Check if it's actually displayed (not just in page source)
            if element.is_displayed():
                print(f'Text "{text}" is now visible!')
                return element

        except selenium.common.exceptions.NoSuchElementException:
            pass  # The text isn't found yet, keep scrolling

        # Scroll down bit-by-bit
        driver.execute_script(f"window.scrollBy(0, {scroll_increment});")
        time.sleep(0.5)  # Allow new content to load
        scroll_attempts += 1

    print(f'Text "{text}" not visible after {max_scrolls} scrolls.')
    return None

# Call the function
scroll_until_text_visible("Shop for a Car")

# Close the driver
driver.quit()
